### Import of all libraries

In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from itertools import combinations
import datetime
import squarify

### Read csv file and basic info of data

In [2]:
df = pd.read_csv('Motor_Vehicle_Collisions_Crashes.csv',dtype={'ZIP CODE':'str'})
pd.set_option('display.max_columns', None)

In [3]:
df.shape

(2093041, 29)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2093041 entries, 0 to 2093040
Data columns (total 29 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   CRASH DATE                     object 
 1   CRASH TIME                     object 
 2   BOROUGH                        object 
 3   ZIP CODE                       object 
 4   LATITUDE                       float64
 5   LONGITUDE                      float64
 6   LOCATION                       object 
 7   ON STREET NAME                 object 
 8   CROSS STREET NAME              object 
 9   OFF STREET NAME                object 
 10  NUMBER OF PERSONS INJURED      float64
 11  NUMBER OF PERSONS KILLED       float64
 12  NUMBER OF PEDESTRIANS INJURED  int64  
 13  NUMBER OF PEDESTRIANS KILLED   int64  
 14  NUMBER OF CYCLIST INJURED      int64  
 15  NUMBER OF CYCLIST KILLED       int64  
 16  NUMBER OF MOTORIST INJURED     int64  
 17  NUMBER OF MOTORIST KILLED      int64  
 18  CO

# Data Cleaning Phase

### Conversion of date and creation of time columns

In [5]:
df['CRASH DATE'] = pd.to_datetime(df['CRASH DATE'])
df['day'] = df['CRASH DATE'].dt.day
df['month'] = df['CRASH DATE'].dt.month
df['year'] = df['CRASH DATE'].dt.year

df[['hour', 'minute']] = df['CRASH TIME'].str.split(':', expand=True).astype(int)


def round_hour(row):
    if row['minute'] >= 30:
        return (row['hour'] + 1) % 24
    else:
        return row['hour']
    
df['rounded_hour'] = df.apply(round_hour, axis=1)

### Rearrange position of columns

In [6]:
col = df.pop('COLLISION_ID')
df.insert(0, col.name, col)

col = df.pop('year')
df.insert(2, col.name, col)

col = df.pop('month')
df.insert(2, col.name, col)

col = df.pop('day')
df.insert(2, col.name, col)

col = df.pop('hour')
df.insert(6, col.name, col)

col = df.pop('minute')
df.insert(6, col.name, col)

col = df.pop('rounded_hour')
df.insert(6, col.name, col)

### Keeping only the decade 2013-2023

In [7]:
df = df[(df['year'] > 2012) & (df['year'] < 2024)]
df.sort_values(by='CRASH DATE',inplace=True)
df.reset_index(drop=True,inplace=True)


### Fixing calculation on person columns

In [8]:
df['NUMBER OF PERSONS INJURED'] = df['NUMBER OF PERSONS INJURED'].fillna(df['NUMBER OF PEDESTRIANS INJURED'] + df['NUMBER OF CYCLIST INJURED'] + df['NUMBER OF MOTORIST INJURED'])
df['NUMBER OF PERSONS KILLED'] = df['NUMBER OF PERSONS KILLED'].fillna(df['NUMBER OF PEDESTRIANS KILLED'] + df['NUMBER OF CYCLIST KILLED'] + df['NUMBER OF MOTORIST KILLED'])

df['NUMBER OF PERSONS INJURED'] = df['NUMBER OF PERSONS INJURED'].astype(int)
df['NUMBER OF PERSONS KILLED'] = df['NUMBER OF PERSONS KILLED'].astype(int)

### Filling all NaN values

In [9]:
u = 'Unspecified'

df['BOROUGH'] = df['BOROUGH'].fillna(u).str.title()
df['ZIP CODE'] = df['ZIP CODE'].fillna(u)
df['LATITUDE'] = df['LATITUDE'].fillna(0)
df['LONGITUDE'] = df['LONGITUDE'].fillna(0)
df['LOCATION'] = df['LOCATION'].fillna('(0,0)')
df['ON STREET NAME'] = df['ON STREET NAME'].fillna(u).str.title()
df['CROSS STREET NAME'] = df['CROSS STREET NAME'].fillna(u).str.title()
df['OFF STREET NAME'] = df['OFF STREET NAME'].fillna(u).str.title()

for i in range(1,6):
    vtc  = 'VEHICLE TYPE CODE ' + str(i)
    ctc =  'CONTRIBUTING FACTOR VEHICLE ' + str(i)
    df[vtc] = df[vtc].fillna(u).str.upper().str.strip()
    df[ctc] = df[ctc].fillna(u)


## Change Vehicle types

In [10]:
uniquevehicles = list(df['VEHICLE TYPE CODE 1'].unique())
uniquevehicles

['TAXI',
 'PASSENGER VEHICLE',
 'SPORT UTILITY / STATION WAGON',
 'UNKNOWN',
 'VAN',
 'MOTORCYCLE',
 'OTHER',
 'LARGE COM VEH(6 OR MORE TIRES)',
 'BUS',
 'PICK-UP TRUCK',
 'LIVERY VEHICLE',
 'SMALL COM VEH(4 TIRES)',
 'AMBULANCE',
 'BICYCLE',
 'FIRE TRUCK',
 'SCOOTER',
 'SEDAN',
 'STATION WAGON/SPORT UTILITY VEHICLE',
 'BIKE',
 'UNSPECIFIED',
 '4 DR SEDAN',
 'PEDICAB',
 'BOX TRUCK',
 'MOTORBIKE',
 'CONVERTIBLE',
 'TRACTOR TRUCK DIESEL',
 '2 DR SEDAN',
 'PK',
 'UTILI',
 'SNOW PLOW',
 'DUMP',
 'SANIT',
 'SCOO',
 'AMBUL',
 'SELF',
 'FLAT BED',
 'MOTORSCOOTER',
 'MOPED',
 'TRACTOR TRUCK GASOLINE',
 'UNKNO',
 'CARRY ALL',
 'FREIG',
 'TOW TRUCK / WRECKER',
 'SCHOOL BUS',
 'TRACT',
 'POSTA',
 '3-DOOR',
 'GARBAGE OR REFUSE',
 'SUBN',
 'LIVESTOCK RACK',
 'WAGON',
 'ARMORED TRUCK',
 'UNK',
 'TANKER',
 'CONCRETE MIXER',
 'TRUCK',
 'TRAIL',
 'BEVERAGE TRUCK',
 'SCOOT',
 'FDNY',
 'ECONO',
 '15 PA',
 'SCHOO',
 'REFRIGERATED VAN',
 'TRL',
 'PICK',
 'FLED',
 'UNKOW',
 'CATER',
 'DELV',
 'NOT I',
 'COM

### Categorize vehicle types

In [11]:
#Changing the pattern variable allows for searching of specific types of vehicles
pattern = r'VAN'
matches = [veh for veh in uniquevehicles if re.search(pattern, veh)]

print(matches)

['VAN', 'REFRIGERATED VAN', 'VANET', 'VAN CAMPER', 'VANETTE', 'VAN E', 'VAN W', 'VAN T', 'VAN/T', 'VANG', 'VAN/B', 'VAN A', 'VAN C', 'VAN (', 'DEPT VAN #', 'RV/VAN', 'FORD VAN', 'WORK VAN', 'CARGO VAN', 'MOVING VAN', 'USPS VAN', 'TRUCK VAN', 'POSTAL VAN', 'VAN TRUCK', 'MARKED VAN', 'VAN/TRUCK', 'EMS/VAN', 'SW/VAN', 'NYPD VAN', 'MINI VAN', 'CAMPER VAN', 'WHITE VAN', 'VAN WH', 'NYCHA VAN', 'USPS  VAN', 'MINIVAN', 'MTA VAN', 'BOX VAN', 'PASS VAN', 'VAN (TRANS']


### Create a mapping dictionary in order to correct misspelled vehicle types

In [12]:
vehicle_type_mapping = {
    'AMBULANCE': 'AMBULANCE',
    'AMBUL' : 'AMBULANCE',
    'AMBU' : 'AMBULANCE',
    'AMBULENCE' : 'AMBULANCE',
    'AMBULETTE' : 'AMBULANCE',
    'AMBULACE' : 'AMBULANCE',
    'AMBULANE' : 'AMBULANCE',
    'AMBALANCE' : 'AMBULANCE',
    'AMBUKANCE' : 'AMBULANCE',
    'AMBULANVE' : 'AMBULANCE',
    'ALMBULANCE' : 'AMBULANCE',
    'PASSENGER VEHICLE' : 'UNSPECIFIED',
    'OTHER' : 'UNSPECIFIED',
    'UNKNOWN' : 'UNSPECIFIED',
    '4 DR SEDAN' : 'SEDAN',
    '2 DR SEDAN' : 'SEDAN',
    '4SEDN' : 'SEDAN',
    'CAR/SUV' : 'SUV',
    'STATION WAGON/SPORT UTILITY VEHICLE': 'SUV',
    'SPORT UTILITY / STATION WAGON' : 'SUV',
    'PEDAL BIKE' : 'BICYCLE',
    'E-BIKE' : 'BICYCLE',
    'E BIKE W P' : 'BICYCLE',
    'E BIKE' : 'BICYCLE',
    'E BIKE UNI' : 'BICYCLE',
    'EBIKE' : 'BICYCLE',
    'E-SCOOTER' : 'BICYCLE',
    'E SCOOTER' : 'BICYCLE',
    'ESCOOTER S' : 'BICYCLE',
    'BIKE' : 'BICYCLE',
    'MINIBIKE' : 'MOTORCYCLE',
    'DIRT BIKE' : 'MOTORCYCLE',
    'MOTORBIKE' : 'MOTORCYCLE',
    'MOTORSCOOTER' : 'MOTORCYCLE',
    'MOTOR SCOO' : 'MOTORCYCLE',
    'E MOTORCYC' : 'MOTORCYCLE', 
    'MOTORSCOOT' : 'MOTORCYCLE',
    'ELE MOTORC' : 'MOTORCYCLE',
    'MOTOR' : 'MOTORCYCLE',
    'SCOOTER' : 'MOTORCYCLE',
    '50CC SCOOT' : 'MOTORCYCLE',
    'GAS SCOOTE' : 'MOTORCYCLE',
    'SCOOTER GA' : 'MOTORCYCLE',
    'MOTORSCOOT' : 'MOTORCYCLE',
    'KICK SCOOT' : 'MOTORCYCLE',
    'PUSH SCOOT' : 'MOTORCYCLE',
    'SCOOT' : 'MOTORCYCLE',
    'YELLOW TAX' : 'TAXI',
    'YELLOW CAB' : 'TAXI',
    'CHASSIS CAB' : 'TRUCK',
    'LARGE COM VEH(6 OR MORE TIRES)' : 'TRUCK',
    'SMALL COM VEH(4 TIRES)' : 'TRUCK',
    'FLAT BED' : 'TRUCK',
    'FLAT RACK' : 'TRUCK',
    'DUMP' : 'TRUCK',
    'PK' : 'PICKUP'
}

### After creating the categories, keep the top 12 and the rest categorize as Unspecified

In [13]:
for i in range(1,6):
    string  = 'VEHICLE TYPE CODE ' + str(i)
    df[string] = df[string].apply(lambda x: 'PICKUP' if 'PICK' in x.upper()  else x)
    df[string] = df[string].apply(lambda x: 'TRUCK' if 'TRU' in x.upper() else x)
    df[string] = df[string].apply(lambda x: 'TRUCK' if 'FIRE' in x.upper() else x)
    df[string] = df[string].apply(lambda x: 'TRUCK' if 'GARBA' in x.upper() else x)
    df[string] = df[string].apply(lambda x: 'BUS' if 'BUS' in x.upper() else x)
    df[string] = df[string].apply(lambda x: 'VAN' if 'VAN' in x.upper() else x)
    df[string] = df[string].map(vehicle_type_mapping).fillna(df[string])
    df[string] = df[string].str.title()

In [14]:
for i in range(1,6):
    string  = 'VEHICLE TYPE CODE ' + str(i)
    allowed_values = list(df[string].value_counts().head(12).index)
    df[string] = df[string].apply(lambda x: x if x in allowed_values else 'Unspecified')

In [15]:
allowed_values = list(df['VEHICLE TYPE CODE 1'].value_counts().head(12).index)
allowed_values

['Sedan',
 'Suv',
 'Unspecified',
 'Taxi',
 'Truck',
 'Pickup',
 'Bus',
 'Van',
 'Bicycle',
 'Motorcycle',
 'Livery Vehicle',
 'Ambulance']

### Change names of columns to increase efficiency

In [16]:
df.rename(columns = {'CRASH DATE' : 'CrashDate',
          'CRASH TIME' : 'CrashTime',
         'BOROUGH': 'Borough',
         'ZIP CODE' : 'ZipCode',
         'LATITUDE' : 'Lat',
         'LONGITUDE' : 'Long',
         'LOCATION' : 'Location',
         'ON STREET NAME' : 'OnStreetName',
         'CROSS STREET NAME' : 'CrossStreetName',
         'OFF STREET NAME' : 'OffStreetName',
         'day' : 'Day',
         'month' : 'Month',
         'year' : 'Year',
         'rounded_hour' : 'RoundedHour',      
         'minute' : 'Minute',     
         'hour' : 'Hour',
         'COLLISION_ID' : 'CollisionID',           
         'NUMBER OF PERSONS INJURED' : 'PersonsInjured',
         'NUMBER OF PERSONS KILLED' : 'PersonsKilled',
         'NUMBER OF PEDESTRIANS INJURED' : 'PedestriansInjured',
         'NUMBER OF PEDESTRIANS KILLED' : 'PedestriansKilled', 
         'NUMBER OF CYCLIST INJURED' : 'CyclistsInjured',
         'NUMBER OF CYCLIST KILLED' : 'CyclistsKilled', 
         'NUMBER OF MOTORIST INJURED' : 'MotoristsInjured', 
         'NUMBER OF MOTORIST KILLED' : 'MotoristsKilled',
         'CONTRIBUTING FACTOR VEHICLE 1' : 'FactorVehicle1', 
         'CONTRIBUTING FACTOR VEHICLE 2' : 'FactorVehicle2',
         'CONTRIBUTING FACTOR VEHICLE 3' : 'FactorVehicle3',
         'CONTRIBUTING FACTOR VEHICLE 4' : 'FactorVehicle4',
         'CONTRIBUTING FACTOR VEHICLE 5' : 'FactorVehicle5',
         'VEHICLE TYPE CODE 1' : 'VehicleType1',
         'VEHICLE TYPE CODE 2' : 'VehicleType2',
         'VEHICLE TYPE CODE 3' : 'VehicleType3',
         'VEHICLE TYPE CODE 4' : 'VehicleType4',
         'VEHICLE TYPE CODE 5' : 'VehicleType5'}, inplace = True)

### Creation of Lighting Condition column

In [17]:
# Convert crash_time to datetime
df['CrashTime'] = pd.to_datetime(df['CrashTime']).dt.time


# Define the time intervals for each lighting condition
time_intervals = [
    (pd.to_datetime('00:00:00').time(), pd.to_datetime('05:00:00').time(), 'Night'),
    (pd.to_datetime('05:00:00').time(), pd.to_datetime('06:00:00').time(), 'Dawn'),
    (pd.to_datetime('06:00:00').time(), pd.to_datetime('17:00:00').time(), 'Day'),
    (pd.to_datetime('17:00:00').time(), pd.to_datetime('18:00:00').time(), 'Dusk'),
    (pd.to_datetime('18:00:00').time(), pd.to_datetime('23:59:59').time(), 'Night')
]

# Function to determine lighting condition using vectorized operations
def determine_lighting_condition(time):
    for start, end, condition in time_intervals:
        if start <= time < end:
            return condition
    return 'Night'  # Fallback to 'Night' if no other condition matches

# Apply the function to the 'time' column
df['LightingCondition'] = df['CrashTime'].apply(determine_lighting_condition)

In [18]:
df[['CollisionID', 'CrashTime', 'LightingCondition']]

,CollisionID,CrashTime,LightingCondition
0,68700,03:30:00,Night
1,2918880,07:24:00,Day
2,86194,04:00:00,Night
3,114369,18:20:00,Night
4,47040,08:44:00,Day
...,...,...,...
1955293,4691995,21:16:00,Night
1955294,4691563,02:55:00,Night
1955295,4692258,11:22:00,Day
1955296,4691263,01:08:00,Night


## Save to file

In [20]:
#df.to_csv('Crashes_FINAL.csv', sep=',', index=False, encoding='utf-8')

### Exclusion of Unspecified bourough

In [ ]:
borough_to_exclude = 'Unspecified'
df1 = df[df['Borough'] != borough_to_exclude]

### Calculations of KPIs

In [ ]:
most_common_time = df['RoundedHour'].mode()[0]
print(f"The time of day with the most crashes in the entire dataset is: {most_common_time}:00")

In [ ]:
most_common_time_per_borough = df1.groupby('Borough')['RoundedHour'].agg(lambda x: x.mode()[0]).reset_index()
print("\nThe time of day with the most crashes in each borough:")
print(most_common_time_per_borough)

## Creation of line chart ( Not used since we decided to do graphs in Power BI)

In [ ]:
# accidents_per_year = df1.groupby(['Year', 'Borough']).size().unstack(fill_value=0)

In [ ]:
# accidents_per_year

In [ ]:
# accidents_per_year.plot(kind='line', figsize=(12, 8), marker='o')

# plt.title('Total Number of Accidents per Year for Each Borough')
# plt.xlabel('Year')
# plt.ylabel('Number of Accidents')
# plt.legend(title='Borough')
# plt.grid(True)
# plt.savefig("lineplot.png")
# plt.show()


In [ ]:
# accidents_per_year = df.groupby('Year').size()
# accidents_per_year

In [ ]:
# ax = accidents_per_year.plot(kind='line', figsize=(12, 8), marker='o')
# plt.title('Total Number of Accidents per Year')
# plt.xlabel('Year')
# plt.ylabel('Number of Accidents')
# # plt.legend(title='Borough')
# plt.grid(True)

# # Set y-axis starting from 0
# plt.ylim(bottom=0)

# # Save the plot as a png file
# plt.savefig("lineplot_excluding_manhattan.png")

# # Show the plot
# plt.show()

In [ ]:
# # Task 1: Time of day with the most crashes in the whole dataset
# crashes_by_hour = df['RoundedHour'].value_counts().sort_index().reset_index()
# crashes_by_hour.columns = ['RoundedHour', 'count']

# plt.figure(figsize=(12, 6))
# sns.barplot(data=crashes_by_hour, x='RoundedHour', y='count', palette='viridis')
# plt.title('Number of Crashes by Hour of the Day')
# plt.xlabel('Hour of the Day')
# plt.ylabel('Number of Crashes')
# plt.xticks(rotation=45)
# plt.show()


In [ ]:
# # Task 2: Time of day with the most crashes in each borough
# plt.figure(figsize=(15, 10))
# g = sns.catplot(data=df1, x='RoundedHour', kind='count', col='Borough', col_wrap=3, palette='viridis', height=4, aspect=1.5)
# g.set_titles("{col_name}")
# g.set_axis_labels("Hour of the Day", "Number of Crashes")
# g.set_xticklabels(rotation=45)
# plt.show()


## Extra commands for reference

In [ ]:
# Extra commands

#df.to_csv('Motor_Vehicle_Collisions_Crashes_v2.csv', sep=',', index=False, encoding='utf-8')

#df['NUMBER OF PERSONS INJURED'].isna().sum()

#df[df['NUMBER OF PERSONS INJURED'].isna()]
#df.isna().sum()

#vehicles = df[df['VEHICLE TYPE CODE 1'].str.startswith('AM')]

#list(df['VEHICLE TYPE CODE 1'].unique())

# mask = df['CONTRIBUTING FACTOR VEHICLE 1'].apply(lambda x: str(x).isdigit())
# df = df[~mask]

# # Replace with actual values to be dropped
# values_to_drop = ['.', '?OMME']

# # Filter the DataFrame to exclude the specified values
# df = df[~df['VEHICLE TYPE CODE 1'].isin(values_to_drop)]

#df['VehicleType2'].value_counts().head(50)
#df['VEHICLE TYPE CODE 1'].value_counts().head(50).sum()
#row = df.iloc[555500]